# 📄 Camunda BPMN Builder — Reference Guide
**Version:** 2025-06-21 (Camunda *model-API 7.23* – IJava)  
**Cells:**  

| Cell | Purpose |
|------|---------|
| **1** | Add Camunda JARs (`%classpath add jar …`) |
| **2** | Paste the _re-usable_ builder (`NodeSpec` + `BpmnBuilderStd`) |
| **3** | Example implementation of the builder |
| **4 … N** | One model per cell — create **nodes / flows / coords**, then call the builder |

---

## 1  What the builder does

| Feature | How it’s done |
|---------|---------------|
| Executable **Process**, **Collaboration**, **Pool**, **Lane** | Created automatically |
| **Nodes** | `Map<String, NodeSpec>` (id → class / label / assignee) |
| **Flows** | `List<String[]>` of `{fromId,toId}` — builder wires `SequenceFlow` safely |
| **Camunda assignee** | `UserTask` + non-null `assignee` → `camunda:assignee` |
| **Pool & lane auto-size** | Tight bbox around node coords + margin. Pool label hidden (lane shows role) |
| **BPMNDI** | One shape per node, one edge per flow (two way-points) |
| **Extensible sizes** | Central `DIM` map per element class |
| **Output** | Fully valid `<fileName>.bpmn` |

---

## 2  API

```java
/* helper */
class NodeSpec {
  final Class<? extends FlowNode> cls;
  final String label;
  final String assignee;
  NodeSpec(Class<? extends FlowNode> c,String l,String a){
    cls=c; label=l; assignee=a;
  }
}

/* builder */
BpmnBuilderStd.generate(
    String fileName,          // also Process & file name
    String poolName,          // pool header (hidden by default)
    String laneName,          // lane label (role)
    Map<String,NodeSpec> nodes,
    List<String[]>        flows,
    Map<String,double[]>  pos  // id → {x,y}
);


In [34]:
%classpath add jar /home/jorick/java-libs/camunda/camunda-bpmn-model-7.23.0.jar
%classpath add jar /home/jorick/java-libs/camunda/camunda-xml-model-7.23.0.jar

In [35]:
// ---------- re-usable builder (fixed) ---------------------------------
import org.camunda.bpm.model.bpmn.Bpmn;
import org.camunda.bpm.model.bpmn.BpmnModelInstance;
import org.camunda.bpm.model.bpmn.instance.*;
import org.camunda.bpm.model.bpmn.instance.bpmndi.*;
import org.camunda.bpm.model.bpmn.instance.dc.Bounds;
import org.camunda.bpm.model.bpmn.instance.di.Waypoint;

import java.util.*;

/* -------------------------------------------------------------------- */
/*  helper class describing a node                                      */
enum TaskType {
  UserTask,
  ManualTask,
  ServiceTask,
  SendTask,
  ReceiveTask
}

static class NodeSpec {
  final Class<? extends FlowNode> cls;
  final String label;
  final String assignee;

  NodeSpec(Class<? extends FlowNode> cls, String label, String assignee){
    this.cls = cls;
    this.label = label;
    this.assignee = assignee;
  }

  static NodeSpec start(String label) {
    return new NodeSpec(StartEvent.class, label, null);
  }

  static NodeSpec end(String label) {
    return new NodeSpec(EndEvent.class, label, null);
  }

  static NodeSpec parallelGateway(String label) {
    return new NodeSpec(ParallelGateway.class, label, null);
  }

  static NodeSpec task(String label, TaskType type, String assignee) {
      switch (type) {
        case ManualTask:   return new NodeSpec(ManualTask.class,  label, null);
        case ServiceTask:  return new NodeSpec(ServiceTask.class, label, null);
        case SendTask:     return new NodeSpec(SendTask.class,    label, null);
        case ReceiveTask:  return new NodeSpec(ReceiveTask.class, label, null);
        default:           return new NodeSpec(UserTask.class,    label, assignee);
      }
    }
}

/* -------------------------------------------------------------------- */
/*  STANDARD BUILDER                                                    */
class BpmnBuilderStd {

  /* canonical sizes */
  private static final Map<Class<?>, int[]> DIM = Map.of(
      StartEvent.class,      new int[]{36,36},
      EndEvent.class,        new int[]{36,36},
      ParallelGateway.class, new int[]{50,50},
      ExclusiveGateway.class, new int[]{50,50},
      InclusiveGateway.class, new int[]{50,50},
      UserTask.class,        new int[]{120,60},
      ServiceTask.class,     new int[]{120,60},
      Task.class,            new int[]{120,60}
  );

  /* MAIN ENTRY ------------------------------------------------------- */
  static void generate(
      String fileName,
      String poolName,
      String laneName,
      Map<String,NodeSpec> nodes,
      List<String[]>        flows,
      Map<String,double[]>  pos) throws Exception {

    /* ===== MODEL ROOT ============================================== */
    BpmnModelInstance m = Bpmn.createEmptyModel();
    Definitions defs = m.newInstance(Definitions.class);
    defs.setTargetNamespace("http://camunda.org/examples");
    m.setDefinitions(defs);

    Collaboration collab = m.newInstance(Collaboration.class);
    collab.setId("Collab_"+fileName);
    defs.addChildElement(collab);

    org.camunda.bpm.model.bpmn.instance.Process proc =
        m.newInstance(org.camunda.bpm.model.bpmn.instance.Process.class);
    proc.setId(fileName);
    proc.setName(nodes.get("processName").label);
    proc.setExecutable(true);
    defs.addChildElement(proc);

    /* ===== POOL + LANE ============================================= */
    Participant pool = m.newInstance(Participant.class);
    pool.setId("Pool_"+poolName.replace(" ","_"));
    // pool.setName(poolName);   // <-- suppress label to avoid duplicate
    pool.setProcess(proc);
    collab.addChildElement(pool);

    LaneSet laneSet = m.newInstance(LaneSet.class);
    laneSet.setId("LaneSet_1");
    proc.addChildElement(laneSet);

    Lane lane = m.newInstance(Lane.class);
    lane.setId("Lane_"+laneName.replace(" ","_"));
    lane.setName(laneName);          // only lane shows text
    laneSet.addChildElement(lane);

    /* ===== FLOW NODES ============================================== */
    Map<String,FlowNode> fn = new LinkedHashMap<>();
    for (var e : nodes.entrySet()){
      if (e.getKey().equals("processName")) continue;
      NodeSpec ns = e.getValue();
      FlowNode n  = (FlowNode) m.newInstance(ns.cls);
      n.setId(e.getKey());
      ((BaseElement)n).setAttributeValue("name", ns.label, false);
      proc.addChildElement(n);
      fn.put(e.getKey(), n);
      lane.getFlowNodeRefs().add(n);
      if (n instanceof UserTask) {
          if (ns.assignee != null) {
            ((UserTask) n).setCamundaAssignee(ns.assignee);
          }
        }
    }

    /* ===== SEQUENCE FLOWS ========================================== */
    for (String[] f : flows)
      connect(proc, fn.get(f[0]), fn.get(f[1]));

    /* ===== BPMNDI ================================================== */
    BpmnDiagram dia = m.newInstance(BpmnDiagram.class);
    dia.setId("Diag1");
    BpmnPlane plane = m.newInstance(BpmnPlane.class);
    plane.setId("Plane1");
    plane.setBpmnElement(collab);
    dia.setBpmnPlane(plane);
    defs.addChildElement(dia);

    /* ---- pool bounds (tight) ---- */
    double minX = Double.MAX_VALUE, minY = Double.MAX_VALUE;
    double maxX = Double.MIN_VALUE, maxY = Double.MIN_VALUE;
    
    for (Map.Entry<String, double[]> entry : pos.entrySet()) {
      String id = entry.getKey();
      double[] p = entry.getValue();
      FlowNode node = fn.get(id);

    
      if (node == null || p == null) continue;
    
      int[] size = DIM.entrySet()
        .stream()
        .filter(e -> e.getKey().isAssignableFrom(node.getClass()))
        .map(Map.Entry::getValue)
        .findFirst()
        .orElse(new int[]{120, 60});
    
      double x = p[0], y = p[1];
      minX = Math.min(minX, x);
      maxX = Math.max(maxX, x + size[0]);
      minY = Math.min(minY, y);
      maxY = Math.max(maxY, y + size[1]);
    }
    double marginX = 60;
    double marginY = 30;
    double poolWidth  = (maxX - minX) + marginX * 2 + 36;
    double poolHeight = (maxY - minY) + marginY * 2;
    
    Bounds pb = m.newInstance(Bounds.class);
    pb.setX(minX - marginX);
    pb.setY(minY - marginY);
    pb.setWidth(poolWidth);
    pb.setHeight(poolHeight);
    
    BpmnShape poolShape = m.newInstance(BpmnShape.class);
    poolShape.setBpmnElement(pool);
    poolShape.setBounds(pb);
    plane.addChildElement(poolShape);

    BpmnShape laneSh = m.newInstance(BpmnShape.class);
    laneSh.setBpmnElement(lane);
    Bounds lb = m.newInstance(Bounds.class);
    lb.setX(pb.getX()); lb.setY(pb.getY());
    lb.setWidth(pb.getWidth()); lb.setHeight(pb.getHeight());
    laneSh.setBounds(lb);
    plane.addChildElement(laneSh);

    /* ---- node shapes ---- */
    for (var e : fn.entrySet()){
      String id = e.getKey();
      FlowNode n = e.getValue();
      double[] p = pos.get(id); if (p==null) continue;

      int[] d = DIM.entrySet()
                   .stream()
                   .filter(en -> en.getKey().isAssignableFrom(n.getClass()))
                   .map(Map.Entry::getValue)
                   .findFirst().orElse(new int[]{120,60});

      BpmnShape s = m.newInstance(BpmnShape.class);
      s.setBpmnElement(n);
      Bounds b = m.newInstance(Bounds.class);
      b.setX(p[0]); b.setY(p[1]); b.setWidth(d[0]); b.setHeight(d[1]);
      s.setBounds(b);
      plane.addChildElement(s);
    }

    /* ---- edges ---- */
    for (SequenceFlow f : m.getModelElementsByType(SequenceFlow.class)){
      BpmnEdge e = m.newInstance(BpmnEdge.class);
      e.setBpmnElement(f);
      double[] s = pos.get(f.getSource().getId());
      double[] t = pos.get(f.getTarget().getId());

      int[] sd = DIM.entrySet().stream()
        .filter(en -> en.getKey().isAssignableFrom(f.getSource().getClass()))
        .findFirst().get().getValue();
      int[] td = DIM.entrySet().stream()
        .filter(en -> en.getKey().isAssignableFrom(f.getTarget().getClass()))
        .findFirst().get().getValue();

      Waypoint w1 = m.newInstance(Waypoint.class);
      w1.setX((int)(s[0]+sd[0])); w1.setY((int)(s[1]+sd[1]/2));
      Waypoint w2 = m.newInstance(Waypoint.class);
      w2.setX((int)(t[0]));       w2.setY((int)(t[1]+td[1]/2));
      e.addChildElement(w1); e.addChildElement(w2);
      plane.addChildElement(e);
    }

    /* ===== SAVE ===================================================== */
    Bpmn.writeModelToFile(new java.io.File(fileName+".bpmn"), m);
    System.out.println("✅  Saved "+fileName+".bpmn");
  }

  /* helper to link two FlowNodes ------------------------------------ */
  private static void connect(org.camunda.bpm.model.bpmn.instance.Process p,
                              FlowNode from, FlowNode to){
    BpmnModelInstance m = (BpmnModelInstance) p.getModelInstance();
    SequenceFlow sf = m.newInstance(SequenceFlow.class);
    sf.setId("flow_"+from.getId()+"_to_"+to.getId());
    p.addChildElement(sf);           // 1⃣ attach to model first
    sf.setSource(from); sf.setTarget(to);      // 2⃣ wire ends
  }
}


In [3]:
// Sample dynamic model creation with role assignments

Map<String, NodeSpec> nodes = Map.ofEntries(
  Map.entry("processName", new NodeSpec(null, "Extend Credit & Invoice Buyer", null)),
  Map.entry("start",       new NodeSpec(StartEvent.class,      "Start Credit Extension", null)),
  Map.entry("record",      new NodeSpec(UserTask.class,        "Record Buyer Details",    "AR Clerk")),
  Map.entry("fork",        new NodeSpec(ParallelGateway.class, "", null)),
  Map.entry("sendApp",     new NodeSpec(UserTask.class,        "Send Credit Application", "AR Clerk")),
  Map.entry("runCheck",    new NodeSpec(ServiceTask.class,     "Run Credit Check",        null)),
  Map.entry("approval",    new NodeSpec(UserTask.class,        "Post-Credit Approval",    "AR Clerk")),
  Map.entry("sendTerms",   new NodeSpec(UserTask.class,        "Send Terms of Sale",      "AR Clerk")),
  Map.entry("join",        new NodeSpec(ParallelGateway.class, "", null)),
  Map.entry("invoice",     new NodeSpec(UserTask.class,        "Send Invoice",            "AR Clerk")),
  Map.entry("copy",        new NodeSpec(UserTask.class,        "Make Second Physical Copy", "AR Clerk")),
  Map.entry("recordSale",  new NodeSpec(UserTask.class,        "Record Sale on Credit",   "AR Clerk")),
  Map.entry("end",         new NodeSpec(EndEvent.class,        "AR Case Closed",          null))
);

List<String[]> flows = List.of(
  new String[]{"start",      "record"},
  new String[]{"record",     "fork"},
  new String[]{"fork",       "sendApp"},
  new String[]{"fork",       "runCheck"},
  new String[]{"fork",       "sendTerms"},
  new String[]{"runCheck",   "approval"},
  new String[]{"sendApp",    "join"},
  new String[]{"approval",   "join"},
  new String[]{"sendTerms",  "join"},
  new String[]{"join",       "invoice"},
  new String[]{"invoice",    "copy"},
  new String[]{"copy",       "recordSale"},
  new String[]{"recordSale", "end"}
);

Map<String, double[]> pos = Map.ofEntries(
  Map.entry("start",      new double[]{  40, 150}),
  Map.entry("record",     new double[]{ 140, 150}),
  Map.entry("fork",       new double[]{ 280, 150}),
  Map.entry("sendApp",    new double[]{ 420,  60}),
  Map.entry("runCheck",   new double[]{ 420, 150}),
  Map.entry("sendTerms",  new double[]{ 420, 240}),
  Map.entry("approval",   new double[]{ 560, 150}),
  Map.entry("join",       new double[]{ 700, 150}),
  Map.entry("invoice",    new double[]{ 840, 150}),
  Map.entry("copy",       new double[]{ 980, 150}),
  Map.entry("recordSale", new double[]{1120, 150}),
  Map.entry("end",        new double[]{1260, 150})
);

// Run generator
BpmnBuilderStd.generate(
  "FTP_199_ar_extend_credit",
  "Accounting",
  "AR Clerk",
  nodes,
  flows,
  pos
);


✅  Saved FTP_199_ar_extend_credit.bpmn


# CEM_110

In [4]:
// Source: CEM_110 INPUT.txt :contentReference[oaicite:0]{index=0}
// Builder API style: METHOD & DOCS.html :contentReference[oaicite:1]{index=1}
/* Use-case 1 ─ Verify signed documents & update CRM */
Map<String, NodeSpec> nodes = Map.ofEntries(
  Map.entry("processName", new NodeSpec(null, "Verify docs & CRM", null)),
  Map.entry("start",       NodeSpec.start("Begin")),
  Map.entry("verifyDocs",  NodeSpec.task("Verify signed documents", TaskType.UserTask, "Account Manager")),
  Map.entry("updateCRM",   NodeSpec.task("Update CRM records", TaskType.ServiceTask, null)),
  Map.entry("end",         NodeSpec.end("Done"))
);
List<String[]> flows = List.of(
  new String[]{"start","verifyDocs"},
  new String[]{"verifyDocs","updateCRM"},
  new String[]{"updateCRM","end"}
);
Map<String,double[]> pos = Map.ofEntries(
  Map.entry("start",     new double[]{ 40,150}),
  Map.entry("verifyDocs",new double[]{140,150}),
  Map.entry("updateCRM", new double[]{280,150}),
  Map.entry("end",       new double[]{420,150})
);
BpmnBuilderStd.generate(
  "CEM_110_uc1_verify_docs_crm",
  "Onboarding",
  "Account Manager",
  nodes, flows, pos
);


✅  Saved CEM_110_uc1_verify_docs_crm.bpmn


In [5]:
// Source: CEM_110 INPUT.txt :contentReference[oaicite:2]{index=2}
/* Use-case 2 ─ Assign work & inform client */
Map<String, NodeSpec> nodes = Map.ofEntries(
  Map.entry("processName", new NodeSpec(null, "Assign work & notify", null)),
  Map.entry("start",       NodeSpec.start("Begin")),
  Map.entry("assignWork",  NodeSpec.task("Assign work to team", TaskType.UserTask, "Account Manager")),
  Map.entry("inform",      NodeSpec.task("Inform client of start", TaskType.SendTask, null)),
  Map.entry("end",         NodeSpec.end("Project started"))
);
List<String[]> flows = List.of(
  new String[]{"start","assignWork"},
  new String[]{"assignWork","inform"},
  new String[]{"inform","end"}
);
Map<String,double[]> pos = Map.ofEntries(
  Map.entry("start",      new double[]{ 40,150}),
  Map.entry("assignWork", new double[]{140,150}),
  Map.entry("inform",     new double[]{280,150}),
  Map.entry("end",        new double[]{420,150})
);
BpmnBuilderStd.generate(
  "CEM_110_uc2_assign_notify",
  "Onboarding",
  "Account Manager",
  nodes, flows, pos
);


✅  Saved CEM_110_uc2_assign_notify.bpmn


In [6]:
// Source: CEM_110 INPUT.txt :contentReference[oaicite:3]{index=3}
/* Use-case 3 ─ Survey & early feedback */
Map<String, NodeSpec> nodes = Map.ofEntries(
  Map.entry("processName", new NodeSpec(null, "Survey & feedback", null)),
  Map.entry("start",  NodeSpec.start("Begin")),
  Map.entry("fork",   NodeSpec.parallelGateway("")),
  Map.entry("survey", NodeSpec.task("Send onboarding survey", TaskType.SendTask, null)),
  Map.entry("change", NodeSpec.task("Apply early feedback", TaskType.UserTask, "Account Manager")),
  Map.entry("join",   NodeSpec.parallelGateway("")),
  Map.entry("end",    NodeSpec.end("Feedback handled"))
);
List<String[]> flows = List.of(
  new String[]{"start","fork"},
  new String[]{"fork","survey"},
  new String[]{"fork","change"},
  new String[]{"survey","join"},
  new String[]{"change","join"},
  new String[]{"join","end"}
);
Map<String,double[]> pos = Map.ofEntries(
  Map.entry("start",  new double[]{ 40,150}),
  Map.entry("fork",   new double[]{140,150}),
  Map.entry("survey", new double[]{280, 90}),
  Map.entry("change", new double[]{280,210}),
  Map.entry("join",   new double[]{420,150}),
  Map.entry("end",    new double[]{560,150})
);
BpmnBuilderStd.generate(
  "CEM_110_uc3_survey_feedback",
  "Onboarding",
  "Account Manager",
  nodes, flows, pos
);


✅  Saved CEM_110_uc3_survey_feedback.bpmn


In [7]:
// Source: CEM_110 INPUT.txt :contentReference[oaicite:4]{index=4}
/* Use-case 4 ─ Assess cancellation impact & contact customer */
Map<String, NodeSpec> nodes = Map.ofEntries(
  Map.entry("processName", new NodeSpec(null, "Assess impact & outreach", null)),
  Map.entry("start",    NodeSpec.start("Begin")),
  Map.entry("assess",   NodeSpec.task("Assess cancellation impact", TaskType.UserTask, "Customer-Success Rep")),
  Map.entry("contact",  NodeSpec.task("Contact customer", TaskType.SendTask, null)),
  Map.entry("record",   NodeSpec.task("Record outreach status", TaskType.UserTask, "Customer-Success Rep")),
  Map.entry("end",      NodeSpec.end("Outreach logged"))
);
List<String[]> flows = List.of(
  new String[]{"start","assess"},
  new String[]{"assess","contact"},
  new String[]{"contact","record"},
  new String[]{"record","end"}
);
Map<String,double[]> pos = Map.ofEntries(
  Map.entry("start",   new double[]{ 40,150}),
  Map.entry("assess",  new double[]{140,150}),
  Map.entry("contact", new double[]{280,150}),
  Map.entry("record",  new double[]{420,150}),
  Map.entry("end",     new double[]{560,150})
);
BpmnBuilderStd.generate(
  "CEM_110_uc4_assess_outreach",
  "Retention",
  "Customer-Success Rep",
  nodes, flows, pos
);


✅  Saved CEM_110_uc4_assess_outreach.bpmn


In [8]:
// Source: CEM_110 INPUT.txt :contentReference[oaicite:5]{index=5}
/* Use-case 5 ─ Run retention conversation & capture learnings */
Map<String, NodeSpec> nodes = Map.ofEntries(
  Map.entry("processName", new NodeSpec(null, "Retention conversation", null)),
  Map.entry("start",     NodeSpec.start("Begin")),
  Map.entry("conversation", NodeSpec.task("Run retention conversation", TaskType.UserTask, "Customer-Success Rep")),
  Map.entry("offer",        NodeSpec.task("Present offer", TaskType.SendTask, null)),
  Map.entry("decision",     NodeSpec.task("Record decision", TaskType.UserTask, "Customer-Success Rep")),
  Map.entry("learnings",    NodeSpec.task("Share learnings", TaskType.ServiceTask, null)),
  Map.entry("end",          NodeSpec.end("Case closed"))
);
List<String[]> flows = List.of(
  new String[]{"start","conversation"},
  new String[]{"conversation","offer"},
  new String[]{"offer","decision"},
  new String[]{"decision","learnings"},
  new String[]{"learnings","end"}
);
Map<String,double[]> pos = Map.ofEntries(
  Map.entry("start",       new double[]{ 40,150}),
  Map.entry("conversation",new double[]{140,150}),
  Map.entry("offer",       new double[]{280,150}),
  Map.entry("decision",    new double[]{420,150}),
  Map.entry("learnings",   new double[]{560,150}),
  Map.entry("end",         new double[]{700,150})
);
BpmnBuilderStd.generate(
  "CEM_110_uc5_retention_call",
  "Retention",
  "Customer-Success Rep",
  nodes, flows, pos
);


✅  Saved CEM_110_uc5_retention_call.bpmn


# CEM_435

In [9]:
// CEM_435 – UC1  :contentReference[oaicite:5]{index=5}
Map<String,NodeSpec> nodes = Map.ofEntries(
  Map.entry("processName", new NodeSpec(null,"Gather Requirements Early",null)),
  Map.entry("start",       new NodeSpec(StartEvent.class,"Client Signed",null)),
  Map.entry("sendQ",       new NodeSpec(UserTask.class,"Send new-client questionnaire","Account Manager")),
  Map.entry("schedule",    new NodeSpec(UserTask.class,"Schedule future calls","Account Manager")),
  Map.entry("end",         new NodeSpec(EndEvent.class,"Next-steps confirmed",null))
);

List<String[]> flows = List.of(
  new String[]{"start","sendQ"},
  new String[]{"sendQ","schedule"},
  new String[]{"schedule","end"}
);

Map<String,double[]> pos = Map.ofEntries(
  Map.entry("start",    new double[]{ 40,150}),
  Map.entry("sendQ",    new double[]{140,150}),
  Map.entry("schedule", new double[]{280,150}),
  Map.entry("end",      new double[]{420,150})
);

BpmnBuilderStd.generate(
  "CEM_435_uc1_gather_requirements",
  "Client Onboarding",
  "Account Manager",
  nodes, flows, pos
);


✅  Saved CEM_435_uc1_gather_requirements.bpmn


In [10]:
// CEM_435 – UC2  :contentReference[oaicite:6]{index=6}
Map<String,NodeSpec> nodes = Map.ofEntries(
  Map.entry("processName", new NodeSpec(null,"Align Expectations at Start",null)),
  Map.entry("start",       new NodeSpec(StartEvent.class,"Kick-off agreed",null)),
  Map.entry("inform",      new NodeSpec(UserTask.class,"Inform client of project start","Account Manager")),
  Map.entry("followUp",    new NodeSpec(UserTask.class,"Hold follow-up call","Account Manager")),
  Map.entry("end",         new NodeSpec(EndEvent.class,"Feedback captured",null))
);

List<String[]> flows = List.of(
  new String[]{"start","inform"},
  new String[]{"inform","followUp"},
  new String[]{"followUp","end"}
);

Map<String,double[]> pos = Map.ofEntries(
  Map.entry("start",    new double[]{ 40,150}),
  Map.entry("inform",   new double[]{140,150}),
  Map.entry("followUp", new double[]{280,150}),
  Map.entry("end",      new double[]{420,150})
);

BpmnBuilderStd.generate(
  "CEM_435_uc2_align_expectations",
  "Client Onboarding",
  "Account Manager",
  nodes, flows, pos
);


✅  Saved CEM_435_uc2_align_expectations.bpmn


In [11]:
// CEM_435 – UC3  :contentReference[oaicite:7]{index=7}
Map<String,NodeSpec> nodes = Map.ofEntries(
  Map.entry("processName", new NodeSpec(null,"Assess Cancellation Risk",null)),
  Map.entry("start",       new NodeSpec(StartEvent.class,"Churn case opened",null)),
  Map.entry("quantify",    new NodeSpec(UserTask.class,"Quantify cancellation impact","Customer-Success Rep")),
  Map.entry("determine",   new NodeSpec(UserTask.class,"Determine ways to increase value","Customer-Success Rep")),
  Map.entry("fork",        new NodeSpec(ParallelGateway.class,"",null)),
  Map.entry("sendEmail",   new NodeSpec(UserTask.class,"Send email to customer","Customer-Success Rep")),
  Map.entry("recordResp",  new NodeSpec(UserTask.class,"Record customer response","Customer-Success Rep")),
  Map.entry("call",        new NodeSpec(UserTask.class,"Call customer","Customer-Success Rep")),
  Map.entry("join",        new NodeSpec(ParallelGateway.class,"",null)),
  Map.entry("end",         new NodeSpec(EndEvent.class,"Initial contact complete",null))
);

List<String[]> flows = List.of(
  new String[]{"start","quantify"},
  new String[]{"quantify","determine"},
  new String[]{"determine","fork"},
  new String[]{"fork","sendEmail"},
  new String[]{"fork","call"},
  new String[]{"sendEmail","recordResp"},
  new String[]{"recordResp","join"},
  new String[]{"call","join"},
  new String[]{"join","end"}
);

Map<String,double[]> pos = Map.ofEntries(
  Map.entry("start",      new double[]{ 40,150}),
  Map.entry("quantify",   new double[]{140,150}),
  Map.entry("determine",  new double[]{280,150}),
  Map.entry("fork",       new double[]{420,150}),
  Map.entry("sendEmail",  new double[]{560, 60}),
  Map.entry("recordResp", new double[]{700, 60}),
  Map.entry("call",       new double[]{560,240}),
  Map.entry("join",       new double[]{840,150}),
  Map.entry("end",        new double[]{980,150})
);

BpmnBuilderStd.generate(
  "CEM_435_uc3_assess_risk",
  "Churn-Avoidance",
  "Customer-Success Rep",
  nodes, flows, pos
);


✅  Saved CEM_435_uc3_assess_risk.bpmn


In [16]:
// CEM_435 – UC4  :contentReference[oaicite:8]{index=8}
Map<String,NodeSpec> nodes = Map.ofEntries(
  Map.entry("processName", new NodeSpec(null,"Improve Retention via Value Conversation",null)),
  Map.entry("start",       new NodeSpec(StartEvent.class,"Customer engaged",null)),
  Map.entry("askQ",        new NodeSpec(UserTask.class,"Ask open-ended questions","Customer-Success Rep")),
  Map.entry("showMetrics", new NodeSpec(UserTask.class,"Inform key metrics","Customer-Success Rep")),
  Map.entry("offer",       new NodeSpec(UserTask.class,"Present special offer","Customer-Success Rep")),
  Map.entry("thank",       new NodeSpec(UserTask.class,"Thank customer","Customer-Success Rep")),
  Map.entry("askConcerns", new NodeSpec(UserTask.class,"Ask for other concerns","Customer-Success Rep")),
  Map.entry("xor",         new NodeSpec(ParallelGateway.class,"",null)),
  Map.entry("sendMgr",     new NodeSpec(UserTask.class,"Send learnings to managers","Customer-Success Rep")),
  Map.entry("note",        new NodeSpec(UserTask.class,"Note down learnings","Customer-Success Rep")),
  Map.entry("end",         new NodeSpec(EndEvent.class,"Case closed",null))
);

List<String[]> flows = List.of(
  new String[]{"start","askQ"},
  new String[]{"askQ","showMetrics"},
  new String[]{"showMetrics","offer"},
  new String[]{"offer","thank"},
  new String[]{"thank","askConcerns"},
  new String[]{"askConcerns","xor"},
  new String[]{"xor","sendMgr"},
  new String[]{"xor","note"},
  new String[]{"sendMgr","end"},
  new String[]{"note","end"}
);

Map<String,double[]> pos = Map.ofEntries(
  Map.entry("start",      new double[]{ 40,150}),
  Map.entry("askQ",       new double[]{140,150}),
  Map.entry("showMetrics",new double[]{280,150}),
  Map.entry("offer",      new double[]{420,150}),
  Map.entry("thank",      new double[]{560,150}),
  Map.entry("askConcerns",new double[]{700,150}),
  Map.entry("xor",        new double[]{840,150}),
  Map.entry("sendMgr",    new double[]{980, 60}),
  Map.entry("note",       new double[]{980,240}),
  Map.entry("end",        new double[]{1120,150})
);

BpmnBuilderStd.generate(
  "CEM_435_uc4_improve_retention",
  "Churn-Avoidance",
  "Customer-Success Rep",
  nodes, flows, pos
);


✅  Saved CEM_435_uc4_improve_retention.bpmn


# CEM_975

In [17]:
// Draft marketing plan & secure approval :contentReference[oaicite:0]{index=0}
Map<String,NodeSpec> nodes = Map.ofEntries(
  Map.entry("processName", new NodeSpec(null, "Draft marketing plan & secure approval", null)),
  Map.entry("start",       new NodeSpec(StartEvent.class,  "",                  null)),
  Map.entry("draftPlan",   new NodeSpec(UserTask.class,   "Draft marketing plan","accountManager")),
  Map.entry("approvePlan", new NodeSpec(UserTask.class,   "Approve plan",        "client")),
  Map.entry("end",         new NodeSpec(EndEvent.class,    "",                  null))
);

List<String[]> flows = List.of(
  new String[]{"start","draftPlan"},
  new String[]{"draftPlan","approvePlan"},
  new String[]{"approvePlan","end"}
);

Map<String,double[]> pos = Map.ofEntries(
  Map.entry("start",       new double[]{  0,150}),
  Map.entry("draftPlan",   new double[]{140,150}),
  Map.entry("approvePlan", new double[]{280,150}),
  Map.entry("end",         new double[]{420,150})
);

BpmnBuilderStd.generate("CEM_975_draft_marketing_plan","Onboarding","Account Manager",nodes,flows,pos);


✅  Saved CEM_975_draft_marketing_plan.bpmn


In [18]:
// Announce project start & allocate team :contentReference[oaicite:1]{index=1}
Map<String,NodeSpec> nodes = Map.ofEntries(
  Map.entry("processName", new NodeSpec(null, "Announce project start & allocate team", null)),
  Map.entry("start",        new NodeSpec(StartEvent.class,   "",                     null)),
  Map.entry("ps",           new NodeSpec(ParallelGateway.class,"",                   null)),
  Map.entry("informClient", new NodeSpec(UserTask.class,     "Inform client of start","accountManager")),
  Map.entry("assignWork",   new NodeSpec(UserTask.class,     "Assign work to team",  "accountManager")),
  Map.entry("pj",           new NodeSpec(ParallelGateway.class,"",                   null)),
  Map.entry("end",          new NodeSpec(EndEvent.class,     "",                     null))
);

List<String[]> flows = List.of(
  new String[]{"start","ps"},
  new String[]{"ps","informClient"},
  new String[]{"ps","assignWork"},
  new String[]{"informClient","pj"},
  new String[]{"assignWork","pj"},
  new String[]{"pj","end"}
);

Map<String,double[]> pos = Map.ofEntries(
  Map.entry("start",        new double[]{  0,150}),
  Map.entry("ps",           new double[]{ 80,150}),
  Map.entry("informClient", new double[]{200, 80}),
  Map.entry("assignWork",   new double[]{200,220}),
  Map.entry("pj",           new double[]{320,150}),
  Map.entry("end",          new double[]{460,150})
);

BpmnBuilderStd.generate("CEM_975_project_start_allocate_team","Onboarding","Account Manager",nodes,flows,pos);


✅  Saved CEM_975_project_start_allocate_team.bpmn


In [21]:
// Week-4 engagement & adjust plan :contentReference[oaicite:2]{index=2}
Map<String,NodeSpec> nodes = Map.ofEntries(
  Map.entry("processName", new NodeSpec(null, "Week-4 engagement & adjust plan", null)),
  Map.entry("start",      new NodeSpec(StartEvent.class,        "",                   null)),
  Map.entry("orSplit",    new NodeSpec(InclusiveGateway.class,  "",                   null)),
  Map.entry("survey",     new NodeSpec(ServiceTask.class,       "Send survey",        null)),
  Map.entry("call",       new NodeSpec(UserTask.class,          "Follow-up call",     "accountManager")),
  Map.entry("adjustPlan", new NodeSpec(UserTask.class,          "Adjust plan",        "accountManager")),
  Map.entry("orJoin",     new NodeSpec(InclusiveGateway.class,  "",                   null)),
  Map.entry("end",        new NodeSpec(EndEvent.class,          "",                   null))
);

List<String[]> flows = List.of(
  new String[]{"start","orSplit"},
  new String[]{"orSplit","survey"},
  new String[]{"survey","orJoin"},
  new String[]{"orSplit","call"},
  new String[]{"call","adjustPlan"},
  new String[]{"adjustPlan","orJoin"},
  new String[]{"orJoin","end"}
);

Map<String,double[]> pos = Map.ofEntries(
  Map.entry("start",      new double[]{  0,150}),
  Map.entry("orSplit",    new double[]{100,150}),
  Map.entry("survey",     new double[]{220, 80}),
  Map.entry("call",       new double[]{220,150}),
  Map.entry("adjustPlan", new double[]{340,150}),
  Map.entry("orJoin",     new double[]{460,150}),
  Map.entry("end",        new double[]{580,150})
);

BpmnBuilderStd.generate("CEM_975_week4_engagement","Onboarding","Account Manager",nodes,flows,pos);


✅  Saved CEM_975_week4_engagement.bpmn


In [22]:
// Analyse usage & contact customer :contentReference[oaicite:3]{index=3}
Map<String,NodeSpec> nodes = Map.ofEntries(
  Map.entry("processName", new NodeSpec(null, "Analyse usage & contact customer", null)),
  Map.entry("start",     new NodeSpec(StartEvent.class, "",                 null)),
  Map.entry("identify",  new NodeSpec(UserTask.class,  "Identify key value","csr")),
  Map.entry("analyse",   new NodeSpec(UserTask.class,  "Analyse usage",     "csr")),
  Map.entry("confirm",   new NodeSpec(UserTask.class,  "Confirm call time", "csr")),
  Map.entry("end",       new NodeSpec(EndEvent.class,  "",                  null))
);

List<String[]> flows = List.of(
  new String[]{"start","identify"},
  new String[]{"identify","analyse"},
  new String[]{"analyse","confirm"},
  new String[]{"confirm","end"}
);

Map<String,double[]> pos = Map.ofEntries(
  Map.entry("start",    new double[]{  0,150}),
  Map.entry("identify", new double[]{140,150}),
  Map.entry("analyse",  new double[]{280,150}),
  Map.entry("confirm",  new double[]{420,150}),
  Map.entry("end",      new double[]{560,150})
);

BpmnBuilderStd.generate("CEM_975_analyse_usage_contact","Success","Customer-Success Rep",nodes,flows,pos);


✅  Saved CEM_975_analyse_usage_contact.bpmn


In [23]:
// Demonstrate value & offer incentives :contentReference[oaicite:4]{index=4}
Map<String,NodeSpec> nodes = Map.ofEntries(
  Map.entry("processName", new NodeSpec(null, "Demonstrate value & offer incentives", null)),
  Map.entry("start",        new NodeSpec(StartEvent.class, "",                  null)),
  Map.entry("questions",    new NodeSpec(UserTask.class,  "Ask open-ended questions","csr")),
  Map.entry("ideas",        new NodeSpec(UserTask.class,  "Describe value ideas",    "csr")),
  Map.entry("metrics",      new NodeSpec(UserTask.class,  "Show key metrics",        "csr")),
  Map.entry("offer",        new NodeSpec(UserTask.class,  "Present special offer",   "csr")),
  Map.entry("recordDecision",new NodeSpec(UserTask.class, "Record customer decision","csr")),
  Map.entry("end",          new NodeSpec(EndEvent.class,  "",                       null))
);

List<String[]> flows = List.of(
  new String[]{"start","questions"},
  new String[]{"questions","ideas"},
  new String[]{"ideas","metrics"},
  new String[]{"metrics","offer"},
  new String[]{"offer","recordDecision"},
  new String[]{"recordDecision","end"}
);

Map<String,double[]> pos = Map.ofEntries(
  Map.entry("start",         new double[]{  0,150}),
  Map.entry("questions",     new double[]{120,150}),
  Map.entry("ideas",         new double[]{260,150}),
  Map.entry("metrics",       new double[]{400,150}),
  Map.entry("offer",         new double[]{540,150}),
  Map.entry("recordDecision",new double[]{680,150}),
  Map.entry("end",           new double[]{820,150})
);

BpmnBuilderStd.generate("CEM_975_demonstrate_value_offer","Success","Customer-Success Rep",nodes,flows,pos);


✅  Saved CEM_975_demonstrate_value_offer.bpmn


# CEM 1000

In [27]:
// Onboard new marketing client – builder cell
// Source: CEM_1000 INPUT.txt :contentReference[oaicite:1]{index=1}
Map<String, NodeSpec> nodes = Map.ofEntries(
  Map.entry("processName",  new NodeSpec(null,               "Onboard new marketing client", null)),
  Map.entry("start",        new NodeSpec(StartEvent.class,   "Start onboarding",             null)),
  Map.entry("draftPlan",    new NodeSpec(UserTask.class,     "Draft marketing plan",         "accountManager")),
  Map.entry("sendForAppr",  new NodeSpec(UserTask.class,     "Send plan for approval",       "accountManager")),
  Map.entry("updateCRM",    new NodeSpec(UserTask.class,     "Update CRM",                   "accountManager")),
  Map.entry("notifyStart",  new NodeSpec(UserTask.class,     "Notify project start",         "accountManager")),
  Map.entry("assignTeam",   new NodeSpec(UserTask.class,     "Assign tasks to team",         "accountManager")),
  Map.entry("applyChanges", new NodeSpec(UserTask.class,     "Apply agreed changes",         "accountManager")),
  Map.entry("end",          new NodeSpec(EndEvent.class,     "Done",                         null))
);

List<String[]> flows = List.of(
  new String[]{"start","draftPlan"},
  new String[]{"draftPlan","sendForAppr"},
  new String[]{"sendForAppr","updateCRM"},
  new String[]{"updateCRM","notifyStart"},
  new String[]{"notifyStart","assignTeam"},
  new String[]{"assignTeam","applyChanges"},
  new String[]{"applyChanges","end"}
);

Map<String,double[]> pos = Map.ofEntries(
  Map.entry("start",        new double[]{ 40,150}),
  Map.entry("draftPlan",    new double[]{140,150}),
  Map.entry("sendForAppr",  new double[]{280,150}),
  Map.entry("updateCRM",    new double[]{420,150}),
  Map.entry("notifyStart",  new double[]{560,150}),
  Map.entry("assignTeam",   new double[]{700,150}),
  Map.entry("applyChanges", new double[]{840,150}),
  Map.entry("end",          new double[]{980,150})
);

BpmnBuilderStd.generate(
  "CEM_1000_onboard_client",
  "Marketing",
  "Account Manager",
  nodes,
  flows,
  pos
);


✅  Saved CEM_1000_onboard_client.bpmn


In [28]:
// Retain at-risk customer – builder cell
// Source: CEM_1000 INPUT.txt :contentReference[oaicite:2]{index=2}
Map<String, NodeSpec> nodes = Map.ofEntries(
  Map.entry("processName",    new NodeSpec(null,             "Retain at-risk customer",          null)),
  Map.entry("start",          new NodeSpec(StartEvent.class, "Start retention",                  null)),
  Map.entry("recordDetails",  new NodeSpec(UserTask.class,   "Record details",                   "customerSuccessRep")),
  Map.entry("determineWays",  new NodeSpec(UserTask.class,   "Determine value increase",         "customerSuccessRep")),
  Map.entry("callCustomer",   new NodeSpec(UserTask.class,   "Call customer",                    "customerSuccessRep")),
  Map.entry("summariseChurn", new NodeSpec(UserTask.class,   "Summarise churn reasons",          "customerSuccessRep")),
  Map.entry("informMetrics",  new NodeSpec(UserTask.class,   "Inform key metrics",               "customerSuccessRep")),
  Map.entry("describeIdeas",  new NodeSpec(UserTask.class,   "Describe value ideas",             "customerSuccessRep")),
  Map.entry("thankTime",      new NodeSpec(UserTask.class,   "Thank customer",                   "customerSuccessRep")),
  Map.entry("askConcerns",    new NodeSpec(UserTask.class,   "Ask other concerns",               "customerSuccessRep")),
  Map.entry("sendLearnings",  new NodeSpec(UserTask.class,   "Send learnings",                   "customerSuccessRep")),
  Map.entry("end",            new NodeSpec(EndEvent.class,   "Case closed",                      null))
);

List<String[]> flows = List.of(
  new String[]{"start","recordDetails"},
  new String[]{"recordDetails","determineWays"},
  new String[]{"determineWays","callCustomer"},
  new String[]{"callCustomer","summariseChurn"},
  new String[]{"summariseChurn","informMetrics"},
  new String[]{"informMetrics","describeIdeas"},
  new String[]{"describeIdeas","thankTime"},
  new String[]{"thankTime","askConcerns"},
  new String[]{"askConcerns","sendLearnings"},
  new String[]{"sendLearnings","end"}
);

Map<String,double[]> pos = Map.ofEntries(
  Map.entry("start",          new double[]{ 40,150}),
  Map.entry("recordDetails",  new double[]{140,150}),
  Map.entry("determineWays",  new double[]{280,150}),
  Map.entry("callCustomer",   new double[]{420,150}),
  Map.entry("summariseChurn", new double[]{560,150}),
  Map.entry("informMetrics",  new double[]{700,150}),
  Map.entry("describeIdeas",  new double[]{840,150}),
  Map.entry("thankTime",      new double[]{980,150}),
  Map.entry("askConcerns",    new double[]{1120,150}),
  Map.entry("sendLearnings",  new double[]{1260,150}),
  Map.entry("end",            new double[]{1400,150})
);

BpmnBuilderStd.generate(
  "CEM_1000_retain_customer",
  "Customer Success",
  "Customer-Success Rep",
  nodes,
  flows,
  pos
);


✅  Saved CEM_1000_retain_customer.bpmn


In [29]:
// Promote new content – builder cell
// Source: CEM_1000 INPUT.txt :contentReference[oaicite:3]{index=3}
Map<String, NodeSpec> nodes = Map.ofEntries(
  Map.entry("processName",   new NodeSpec(null,             "Promote new content", null)),
  Map.entry("start",         new NodeSpec(StartEvent.class, "Start promotion",     null)),
  Map.entry("prepSummary",   new NodeSpec(UserTask.class,   "Prepare summary",     "contentMarketer")),
  Map.entry("outreach",      new NodeSpec(UserTask.class,   "Targeted outreach",   "contentMarketer")),
  Map.entry("end",           new NodeSpec(EndEvent.class,   "Outreach logged",     null))
);

List<String[]> flows = List.of(
  new String[]{"start","prepSummary"},
  new String[]{"prepSummary","outreach"},
  new String[]{"outreach","end"}
);

Map<String,double[]> pos = Map.ofEntries(
  Map.entry("start",       new double[]{40,150}),
  Map.entry("prepSummary", new double[]{140,150}),
  Map.entry("outreach",    new double[]{280,150}),
  Map.entry("end",         new double[]{420,150})
);

BpmnBuilderStd.generate(
  "CEM_1000_promote_content",
  "Content Marketing",
  "Content Marketer",
  nodes,
  flows,
  pos
);


✅  Saved CEM_1000_promote_content.bpmn


In [30]:
// Submit to link round-ups – builder cell
// Source: CEM_1000 INPUT.txt :contentReference[oaicite:4]{index=4}
Map<String, NodeSpec> nodes = Map.ofEntries(
  Map.entry("processName",   new NodeSpec(null,             "Submit to link round-ups", null)),
  Map.entry("start",         new NodeSpec(StartEvent.class, "Start submission",         null)),
  Map.entry("prepSummary",   new NodeSpec(UserTask.class,   "Prepare summary",          "contentMarketer")),
  Map.entry("submitRoundup", new NodeSpec(UserTask.class,   "Submit to round-ups",      "contentMarketer")),
  Map.entry("end",           new NodeSpec(EndEvent.class,   "Backlinks tracked",        null))
);

List<String[]> flows = List.of(
  new String[]{"start","prepSummary"},
  new String[]{"prepSummary","submitRoundup"},
  new String[]{"submitRoundup","end"}
);

Map<String,double[]> pos = Map.ofEntries(
  Map.entry("start",        new double[]{40,150}),
  Map.entry("prepSummary",  new double[]{140,150}),
  Map.entry("submitRoundup",new double[]{280,150}),
  Map.entry("end",          new double[]{420,150})
);

BpmnBuilderStd.generate(
  "CEM_1000_submit_roundups",
  "Content Marketing",
  "Content Marketer",
  nodes,
  flows,
  pos
);


✅  Saved CEM_1000_submit_roundups.bpmn


# FTP_1071

In [41]:
/* ================= Post-credit approval ================= */

Map<String, NodeSpec> nodes = Map.ofEntries(
  Map.entry("processName", new NodeSpec(null, "Post Credit Approval", null)),
  Map.entry("start",     new NodeSpec(StartEvent.class, "Buyer Entered", null)),
  Map.entry("record",    new NodeSpec(UserTask.class,  "Record Buyer Details",     "AR Clerk")),
  Map.entry("approval",  new NodeSpec(UserTask.class,  "Post-Credit Check Approval","AR Clerk")),
  Map.entry("end",       new NodeSpec(EndEvent.class,  "Ready for Invoicing", null))
);

List<String[]> flows = List.of(
  new String[]{"start","record"},
  new String[]{"record","approval"},
  new String[]{"approval","end"}
);

Map<String,double[]> pos = Map.ofEntries(
  Map.entry("start",   new double[]{ 40,150}),
  Map.entry("record",  new double[]{140,150}),
  Map.entry("approval",new double[]{280,150}),
  Map.entry("end",     new double[]{420,150})
);

BpmnBuilderStd.generate(
  "FTP_1071_post_credit_approval",
  "Accounting",
  "AR Clerk",
  nodes,
  flows,
  pos
);


✅  Saved FTP_1071_post_credit_approval.bpmn


In [42]:
/* ================= Invoice or verify terms ================= */

Map<String, NodeSpec> nodes = Map.ofEntries(
  Map.entry("processName",  new NodeSpec(null, "Invoice Or Verify Terms", null)),
  Map.entry("start",        new NodeSpec(StartEvent.class,      "Choose Path", null)),
  Map.entry("decide",       new NodeSpec(ExclusiveGateway.class,"", null)),
  Map.entry("checkTerms",   new NodeSpec(UserTask.class,  "Check Payment Terms",          "AR Clerk")),
  Map.entry("genInvoice",   new NodeSpec(UserTask.class,  "Generate Invoice",             "AR Clerk")),
  Map.entry("sendInvoice",  new NodeSpec(UserTask.class,  "Send Invoice (Software)",      "AR Clerk")),
  Map.entry("logActivity",  new NodeSpec(UserTask.class,  "Record Activity Digitally",    "AR Clerk")),
  Map.entry("monthlyContact",new NodeSpec(UserTask.class,"Establish Monthly Contact",     "AR Clerk")),
  Map.entry("join",         new NodeSpec(ExclusiveGateway.class,"", null)),
  Map.entry("end",          new NodeSpec(EndEvent.class,        "Logged", null))
);

List<String[]> flows = List.of(
  new String[]{"start","decide"},
  new String[]{"decide","checkTerms"},
  new String[]{"decide","genInvoice"},
  new String[]{"decide","sendInvoice"},
  new String[]{"decide","logActivity"},
  new String[]{"decide","monthlyContact"},
  new String[]{"checkTerms","join"},
  new String[]{"genInvoice","join"},
  new String[]{"sendInvoice","join"},
  new String[]{"logActivity","join"},
  new String[]{"monthlyContact","join"},
  new String[]{"join","end"}
);

Map<String,double[]> pos = Map.ofEntries(
  Map.entry("start",          new double[]{ 40,150}),
  Map.entry("decide",         new double[]{140,150}),
  Map.entry("checkTerms",     new double[]{280, 50}),
  Map.entry("genInvoice",     new double[]{280,100}),
  Map.entry("sendInvoice",    new double[]{280,150}),
  Map.entry("logActivity",    new double[]{280,200}),
  Map.entry("monthlyContact", new double[]{280,250}),
  Map.entry("join",           new double[]{420,150}),
  Map.entry("end",            new double[]{560,150})
);

BpmnBuilderStd.generate(
  "FTP_1071_invoice_or_verify_terms",
  "Accounting",
  "AR Clerk",
  nodes,
  flows,
  pos
);


✅  Saved FTP_1071_invoice_or_verify_terms.bpmn


In [43]:
/* ================= Record credit sales ================= */

Map<String, NodeSpec> nodes = Map.ofEntries(
  Map.entry("processName", new NodeSpec(null, "Record Credit Sales", null)),
  Map.entry("start",       new NodeSpec(StartEvent.class, "Begin", null)),
  Map.entry("goods",       new NodeSpec(UserTask.class,  "Record Credit Sale (Goods)",    "AR Clerk")),
  Map.entry("services",    new NodeSpec(UserTask.class,  "Record Credit Sale (Services)", "AR Clerk")),
  Map.entry("end",         new NodeSpec(EndEvent.class,  "AR Updated", null))
);

List<String[]> flows = List.of(
  new String[]{"start","goods"},
  new String[]{"goods","services"},
  new String[]{"services","end"}
);

Map<String,double[]> pos = Map.ofEntries(
  Map.entry("start",    new double[]{ 40,150}),
  Map.entry("goods",    new double[]{140,150}),
  Map.entry("services", new double[]{280,150}),
  Map.entry("end",      new double[]{420,150})
);

BpmnBuilderStd.generate(
  "FTP_1071_record_credit_sales",
  "Accounting",
  "AR Clerk",
  nodes,
  flows,
  pos
);


✅  Saved FTP_1071_record_credit_sales.bpmn


In [44]:
/* ================= Receiving report & correct errors ================= */

Map<String, NodeSpec> nodes = Map.ofEntries(
  Map.entry("processName", new NodeSpec(null, "Receiving Report & Correct Errors", null)),
  Map.entry("start",       new NodeSpec(StartEvent.class,      "Goods Received", null)),
  Map.entry("sendToRec",   new NodeSpec(UserTask.class,        "Send To Receiving Dept.", "AP Specialist")),
  Map.entry("createReport",new NodeSpec(UserTask.class,        "Create Receiving Report", "AP Specialist")),
  Map.entry("decide",      new NodeSpec(ExclusiveGateway.class,"", null)),
  Map.entry("reportErr",   new NodeSpec(UserTask.class,        "Report Errors",           "AP Specialist")),
  Map.entry("confirmDocs", new NodeSpec(UserTask.class,        "Confirm Docs Amended",    "AP Specialist")),
  Map.entry("join",        new NodeSpec(ExclusiveGateway.class,"", null)),
  Map.entry("end",         new NodeSpec(EndEvent.class,        "Documents Validated", null))
);

List<String[]> flows = List.of(
  new String[]{"start","sendToRec"},
  new String[]{"sendToRec","createReport"},
  new String[]{"createReport","decide"},
  new String[]{"decide","reportErr"},
  new String[]{"decide","confirmDocs"},
  new String[]{"reportErr","join"},
  new String[]{"confirmDocs","join"},
  new String[]{"join","end"}
);

Map<String,double[]> pos = Map.ofEntries(
  Map.entry("start",        new double[]{ 40,150}),
  Map.entry("sendToRec",    new double[]{140,150}),
  Map.entry("createReport", new double[]{280,150}),
  Map.entry("decide",       new double[]{420,150}),
  Map.entry("reportErr",    new double[]{560,100}),
  Map.entry("confirmDocs",  new double[]{560,200}),
  Map.entry("join",         new double[]{700,150}),
  Map.entry("end",          new double[]{840,150})
);

BpmnBuilderStd.generate(
  "FTP_1071_receiving_report_correct_errors",
  "Accounting",
  "AP Specialist",
  nodes,
  flows,
  pos
);


✅  Saved FTP_1071_receiving_report_correct_errors.bpmn


In [45]:
/* ================= Schedule payment or book invoice ================= */

Map<String, NodeSpec> nodes = Map.ofEntries(
  Map.entry("processName", new NodeSpec(null, "Schedule Payment Or Book Invoice", null)),
  Map.entry("start",       new NodeSpec(StartEvent.class,      "Invoice Received", null)),
  Map.entry("decide",      new NodeSpec(ExclusiveGateway.class,"", null)),
  Map.entry("schedulePay", new NodeSpec(UserTask.class,        "Schedule Payment", "AP Specialist")),
  Map.entry("enterInvoice",new NodeSpec(UserTask.class,        "Enter Invoice",    "AP Specialist")),
  Map.entry("join",        new NodeSpec(ExclusiveGateway.class,"", null)),
  Map.entry("end",         new NodeSpec(EndEvent.class,        "Invoice Cleared", null))
);

List<String[]> flows = List.of(
  new String[]{"start","decide"},
  new String[]{"decide","schedulePay"},
  new String[]{"decide","enterInvoice"},
  new String[]{"schedulePay","join"},
  new String[]{"enterInvoice","join"},
  new String[]{"join","end"}
);

Map<String,double[]> pos = Map.ofEntries(
  Map.entry("start",        new double[]{ 40,150}),
  Map.entry("decide",       new double[]{140,150}),
  Map.entry("schedulePay",  new double[]{280,100}),
  Map.entry("enterInvoice", new double[]{280,200}),
  Map.entry("join",         new double[]{420,150}),
  Map.entry("end",          new double[]{560,150})
);

BpmnBuilderStd.generate(
  "FTP_1071_schedule_payment_or_book_invoice",
  "Accounting",
  "AP Specialist",
  nodes,
  flows,
  pos
);


✅  Saved FTP_1071_schedule_payment_or_book_invoice.bpmn


In [46]:
/* ================= Verify application completeness ================= */

Map<String, NodeSpec> nodes = Map.ofEntries(
  Map.entry("processName",  new NodeSpec(null, "Verify Application Completeness", null)),
  Map.entry("start",        new NodeSpec(StartEvent.class,      "ID Proof Submitted", null)),
  Map.entry("receiveApp",   new NodeSpec(UserTask.class,        "Receive Application",       "Loan Officer")),
  Map.entry("completeApp",  new NodeSpec(UserTask.class,        "Complete Application",      "Loan Officer")),
  Map.entry("decide",       new NodeSpec(ExclusiveGateway.class,"", null)),
  Map.entry("checkComp",    new NodeSpec(UserTask.class,        "Check Completeness",        "Loan Officer")),
  Map.entry("notice",       new NodeSpec(UserTask.class,        "Give Written Notice",       "Loan Officer")),
  Map.entry("returnApp",    new NodeSpec(UserTask.class,        "Return Application",        "Loan Officer")),
  Map.entry("join",         new NodeSpec(ExclusiveGateway.class,"", null)),
  Map.entry("end",          new NodeSpec(EndEvent.class,        "Ready For Underwriting", null))
);

List<String[]> flows = List.of(
  new String[]{"start","receiveApp"},
  new String[]{"receiveApp","completeApp"},
  new String[]{"completeApp","decide"},
  new String[]{"decide","checkComp"},
  new String[]{"decide","notice"},
  new String[]{"decide","returnApp"},
  new String[]{"checkComp","join"},
  new String[]{"notice","join"},
  new String[]{"returnApp","join"},
  new String[]{"join","end"}
);

Map<String,double[]> pos = Map.ofEntries(
  Map.entry("start",       new double[]{ 40,150}),
  Map.entry("receiveApp",  new double[]{140,150}),
  Map.entry("completeApp", new double[]{280,150}),
  Map.entry("decide",      new double[]{420,150}),
  Map.entry("checkComp",   new double[]{560, 80}),
  Map.entry("notice",      new double[]{560,150}),
  Map.entry("returnApp",   new double[]{560,220}),
  Map.entry("join",        new double[]{700,150}),
  Map.entry("end",         new double[]{840,150})
);

BpmnBuilderStd.generate(
  "FTP_1071_verify_application_completeness",
  "Lending",
  "Loan Officer",
  nodes,
  flows,
  pos
);


✅  Saved FTP_1071_verify_application_completeness.bpmn


In [47]:
/* ================= Underwrite and adjust rate ================= */

Map<String, NodeSpec> nodes = Map.ofEntries(
  Map.entry("processName", new NodeSpec(null, "Underwrite And Adjust Rate", null)),
  Map.entry("start",       new NodeSpec(StartEvent.class, "Begin", null)),
  Map.entry("underwrite",  new NodeSpec(UserTask.class,  "Begin Underwriting", "Loan Officer")),
  Map.entry("changeRate",  new NodeSpec(UserTask.class,  "Change Interest Rate", "Loan Officer")),
  Map.entry("end",         new NodeSpec(EndEvent.class,  "Offer Aligned", null))
);

List<String[]> flows = List.of(
  new String[]{"start","underwrite"},
  new String[]{"underwrite","changeRate"},
  new String[]{"changeRate","end"}
);

Map<String,double[]> pos = Map.ofEntries(
  Map.entry("start",      new double[]{ 40,150}),
  Map.entry("underwrite", new double[]{140,150}),
  Map.entry("changeRate", new double[]{280,150}),
  Map.entry("end",        new double[]{420,150})
);

BpmnBuilderStd.generate(
  "FTP_1071_underwrite_and_adjust_rate",
  "Lending",
  "Loan Officer",
  nodes,
  flows,
  pos
);


✅  Saved FTP_1071_underwrite_and_adjust_rate.bpmn


In [48]:
/* ================= Disburse loan funds ================= */

Map<String, NodeSpec> nodes = Map.ofEntries(
  Map.entry("processName", new NodeSpec(null, "Disburse Loan Funds", null)),
  Map.entry("start",       new NodeSpec(StartEvent.class, "Loan Approved", null)),
  Map.entry("disburse",    new NodeSpec(UserTask.class,  "Receive & Disburse Funds", "Loan Officer")),
  Map.entry("end",         new NodeSpec(EndEvent.class,  "Loan Funded", null))
);

List<String[]> flows = List.of(
  new String[]{"start","disburse"},
  new String[]{"disburse","end"}
);

Map<String,double[]> pos = Map.ofEntries(
  Map.entry("start",    new double[]{ 40,150}),
  Map.entry("disburse", new double[]{140,150}),
  Map.entry("end",      new double[]{280,150})
);

BpmnBuilderStd.generate(
  "FTP_1071_disburse_loan_funds",
  "Lending",
  "Loan Officer",
  nodes,
  flows,
  pos
);


✅  Saved FTP_1071_disburse_loan_funds.bpmn


# FTP_1150

In [49]:
// UC1 – Collect and vet buyer credit
Map<String, NodeSpec> nodes = Map.ofEntries(
  Map.entry("processName", new NodeSpec(null, "Collect & Vet Buyer Credit", null)),
  Map.entry("start",       new NodeSpec(StartEvent.class,      "Start", null)),
  Map.entry("record",      new NodeSpec(UserTask.class,        "Record Buyer Details",   "ar_clerk")),
  Map.entry("fork",        new NodeSpec(ParallelGateway.class, "+", null)),
  Map.entry("sendApp",     new NodeSpec(UserTask.class,        "Send Credit Application","ar_clerk")),
  Map.entry("checkTerms",  new NodeSpec(UserTask.class,        "Check Payment Terms",    "ar_clerk")),
  Map.entry("join",        new NodeSpec(ParallelGateway.class, "+", null)),
  Map.entry("evaluate",    new NodeSpec(ServiceTask.class,     "Evaluate Credit Risk",   null)),
  Map.entry("end",         new NodeSpec(EndEvent.class,        "Credit Evaluated", null))
);

List<String[]> flows = List.of(
  new String[]{"start","record"},
  new String[]{"record","fork"},
  new String[]{"fork","sendApp"},
  new String[]{"fork","checkTerms"},
  new String[]{"sendApp","join"},
  new String[]{"checkTerms","join"},
  new String[]{"join","evaluate"},
  new String[]{"evaluate","end"}
);

Map<String,double[]> pos = Map.ofEntries(
  Map.entry("start",      new double[]{40,150}),
  Map.entry("record",     new double[]{140,150}),
  Map.entry("fork",       new double[]{280,150}),
  Map.entry("sendApp",    new double[]{420, 60}),
  Map.entry("checkTerms", new double[]{420,150}),
  Map.entry("join",       new double[]{560,150}),
  Map.entry("evaluate",   new double[]{700,150}),
  Map.entry("end",        new double[]{840,150})
);

BpmnBuilderStd.generate(
  "FTP_1150_uc1_collect_buyer_credit",
  "Accounting",
  "AR Clerk",
  nodes,
  flows,
  pos
);


✅  Saved FTP_1150_uc1_collect_buyer_credit.bpmn


In [50]:
// UC2 – Maintain account records
Map<String, NodeSpec> nodes = Map.ofEntries(
  Map.entry("processName", new NodeSpec(null, "Maintain Account Records", null)),
  Map.entry("start",   new NodeSpec(StartEvent.class,"Start",null)),
  Map.entry("contact", new NodeSpec(UserTask.class, "Establish Monthly Contact","ar_clerk")),
  Map.entry("copy",    new NodeSpec(UserTask.class, "Make Physical Copy","ar_clerk")),
  Map.entry("digital", new NodeSpec(UserTask.class, "Record Activity Digitally","ar_clerk")),
  Map.entry("end",     new NodeSpec(EndEvent.class, "Records Updated",null))
);

List<String[]> flows = List.of(
  new String[]{"start","contact"},
  new String[]{"contact","copy"},
  new String[]{"copy","digital"},
  new String[]{"digital","end"}
);

Map<String,double[]> pos = Map.ofEntries(
  Map.entry("start",   new double[]{40,150}),
  Map.entry("contact", new double[]{140,150}),
  Map.entry("copy",    new double[]{280,150}),
  Map.entry("digital", new double[]{420,150}),
  Map.entry("end",     new double[]{560,150})
);

BpmnBuilderStd.generate(
  "FTP_1150_uc2_maintain_account_records",
  "Accounting",
  "AR Clerk",
  nodes,
  flows,
  pos
);


✅  Saved FTP_1150_uc2_maintain_account_records.bpmn


In [51]:
// UC3 – Record service sales
Map<String, NodeSpec> nodes = Map.ofEntries(
  Map.entry("processName", new NodeSpec(null, "Record Service Sales on Credit", null)),
  Map.entry("start",  new NodeSpec(StartEvent.class,"Start",null)),
  Map.entry("record", new NodeSpec(UserTask.class,"Record Service Sales","ar_clerk")),
  Map.entry("end",    new NodeSpec(EndEvent.class,"Sales Recorded",null))
);

List<String[]> flows = List.of(
  new String[]{"start","record"},
  new String[]{"record","end"}
);

Map<String,double[]> pos = Map.ofEntries(
  Map.entry("start",  new double[]{40,150}),
  Map.entry("record", new double[]{140,150}),
  Map.entry("end",    new double[]{280,150})
);

BpmnBuilderStd.generate(
  "FTP_1150_uc3_record_service_sales",
  "Accounting",
  "AR Clerk",
  nodes,
  flows,
  pos
);


✅  Saved FTP_1150_uc3_record_service_sales.bpmn


In [52]:
// UC4 – Create receiving paperwork & validate docs
Map<String, NodeSpec> nodes = Map.ofEntries(
  Map.entry("processName", new NodeSpec(null,"Create Receiving Paperwork & Validate Docs",null)),
  Map.entry("start",      new NodeSpec(StartEvent.class,"Start",null)),
  Map.entry("enterPO",    new NodeSpec(UserTask.class,"Enter PO Details","ap_specialist")),
  Map.entry("recvReport", new NodeSpec(UserTask.class,"Create Receiving Report","ap_specialist")),
  Map.entry("invoice",    new NodeSpec(UserTask.class,"Gather Vendor Invoice","ap_specialist")),
  Map.entry("end",        new NodeSpec(EndEvent.class,"Docs Ready",null))
);

List<String[]> flows = List.of(
  new String[]{"start","enterPO"},
  new String[]{"enterPO","recvReport"},
  new String[]{"recvReport","invoice"},
  new String[]{"invoice","end"}
);

Map<String,double[]> pos = Map.ofEntries(
  Map.entry("start",      new double[]{40,150}),
  Map.entry("enterPO",    new double[]{140,150}),
  Map.entry("recvReport", new double[]{280,150}),
  Map.entry("invoice",    new double[]{420,150}),
  Map.entry("end",        new double[]{560,150})
);

BpmnBuilderStd.generate(
  "FTP_1150_uc4_create_receive_docs",
  "Accounting",
  "AP Specialist",
  nodes,
  flows,
  pos
);


✅  Saved FTP_1150_uc4_create_receive_docs.bpmn


In [53]:
// UC5 – Stamp, check dates & schedule payment
Map<String, NodeSpec> nodes = Map.ofEntries(
  Map.entry("processName", new NodeSpec(null,"Stamp, Check Dates & Schedule Payment",null)),
  Map.entry("start",    new NodeSpec(StartEvent.class,"Start",null)),
  Map.entry("stamp",    new NodeSpec(UserTask.class,"Stamp Documents","ap_specialist")),
  Map.entry("check",    new NodeSpec(UserTask.class,"Check Dates","ap_specialist")),
  Map.entry("schedule", new NodeSpec(UserTask.class,"Schedule Payment","ap_specialist")),
  Map.entry("end",      new NodeSpec(EndEvent.class,"Payment Queued",null))
);

List<String[]> flows = List.of(
  new String[]{"start","stamp"},
  new String[]{"stamp","check"},
  new String[]{"check","schedule"},
  new String[]{"schedule","end"}
);

Map<String,double[]> pos = Map.ofEntries(
  Map.entry("start",    new double[]{40,150}),
  Map.entry("stamp",    new double[]{140,150}),
  Map.entry("check",    new double[]{280,150}),
  Map.entry("schedule", new double[]{420,150}),
  Map.entry("end",      new double[]{560,150})
);

BpmnBuilderStd.generate(
  "FTP_1150_uc5_stamp_check_schedule",
  "Accounting",
  "AP Specialist",
  nodes,
  flows,
  pos
);


✅  Saved FTP_1150_uc5_stamp_check_schedule.bpmn


In [54]:
// UC6 – Accept and verify loan application
Map<String, NodeSpec> nodes = Map.ofEntries(
  Map.entry("processName", new NodeSpec(null,"Accept & Verify Loan Application",null)),
  Map.entry("start",       new NodeSpec(StartEvent.class,"Start",null)),
  Map.entry("completeApp", new NodeSpec(UserTask.class,"Complete Loan Application","loan_officer")),
  Map.entry("checkComp",   new NodeSpec(UserTask.class,"Check Completeness","loan_officer")),
  Map.entry("decide",      new NodeSpec(ExclusiveGateway.class,"?",null)),
  Map.entry("notice",      new NodeSpec(UserTask.class,"Give Written Notice","loan_officer")),
  Map.entry("returnApp",   new NodeSpec(UserTask.class,"Return Application","loan_officer")),
  Map.entry("merge",       new NodeSpec(ExclusiveGateway.class,"",null)),
  Map.entry("end",         new NodeSpec(EndEvent.class,"File Ready",null))
);

List<String[]> flows = List.of(
  new String[]{"start","completeApp"},
  new String[]{"completeApp","checkComp"},
  new String[]{"checkComp","decide"},
  new String[]{"decide","notice"},
  new String[]{"decide","returnApp"},
  new String[]{"notice","merge"},
  new String[]{"returnApp","merge"},
  new String[]{"merge","end"}
);

Map<String,double[]> pos = Map.ofEntries(
  Map.entry("start",       new double[]{40,150}),
  Map.entry("completeApp", new double[]{140,150}),
  Map.entry("checkComp",   new double[]{280,150}),
  Map.entry("decide",      new double[]{420,150}),
  Map.entry("notice",      new double[]{560, 60}),
  Map.entry("returnApp",   new double[]{560,150}),
  Map.entry("merge",       new double[]{700,150}),
  Map.entry("end",         new double[]{840,150})
);

BpmnBuilderStd.generate(
  "FTP_1150_uc6_accept_verify_application",
  "Loan Processing",
  "Loan Officer",
  nodes,
  flows,
  pos
);


✅  Saved FTP_1150_uc6_accept_verify_application.bpmn


In [55]:
// UC7 – Underwrite and adjust loan
Map<String, NodeSpec> nodes = Map.ofEntries(
  Map.entry("processName", new NodeSpec(null,"Underwrite & Adjust Loan",null)),
  Map.entry("start",       new NodeSpec(StartEvent.class,"Start",null)),
  Map.entry("beginUW",     new NodeSpec(ServiceTask.class,"Begin Underwriting","loan_officer")),
  Map.entry("checkCred",   new NodeSpec(ServiceTask.class,"Check Credit & Risk","loan_officer")),
  Map.entry("adjust",      new NodeSpec(UserTask.class,"Adjust Loan Amount","loan_officer")),
  Map.entry("end",         new NodeSpec(EndEvent.class,"Terms Finalised",null))
);

List<String[]> flows = List.of(
  new String[]{"start","beginUW"},
  new String[]{"beginUW","checkCred"},
  new String[]{"checkCred","adjust"},
  new String[]{"adjust","end"}
);

Map<String,double[]> pos = Map.ofEntries(
  Map.entry("start",     new double[]{40,150}),
  Map.entry("beginUW",   new double[]{140,150}),
  Map.entry("checkCred", new double[]{280,150}),
  Map.entry("adjust",    new double[]{420,150}),
  Map.entry("end",       new double[]{560,150})
);

BpmnBuilderStd.generate(
  "FTP_1150_uc7_underwrite_adjust_loan",
  "Loan Processing",
  "Loan Officer",
  nodes,
  flows,
  pos
);


✅  Saved FTP_1150_uc7_underwrite_adjust_loan.bpmn


In [56]:
// UC8 – Disburse loan funds
Map<String, NodeSpec> nodes = Map.ofEntries(
  Map.entry("processName", new NodeSpec(null,"Disburse Loan Funds",null)),
  Map.entry("start",    new NodeSpec(StartEvent.class,"Start",null)),
  Map.entry("receive",  new NodeSpec(ServiceTask.class,"Receive Loan Funds","loan_officer")),
  Map.entry("disburse", new NodeSpec(UserTask.class,"Disburse Funds","loan_officer")),
  Map.entry("end",      new NodeSpec(EndEvent.class,"Loan Funded",null))
);

List<String[]> flows = List.of(
  new String[]{"start","receive"},
  new String[]{"receive","disburse"},
  new String[]{"disburse","end"}
);

Map<String,double[]> pos = Map.ofEntries(
  Map.entry("start",    new double[]{40,150}),
  Map.entry("receive",  new double[]{140,150}),
  Map.entry("disburse", new double[]{280,150}),
  Map.entry("end",      new double[]{420,150})
);

BpmnBuilderStd.generate(
  "FTP_1150_uc8_disburse_funds",
  "Loan Processing",
  "Loan Officer",
  nodes,
  flows,
  pos
);


✅  Saved FTP_1150_uc8_disburse_funds.bpmn


In [57]:
// UC9 – Create requisition and collect approvals
Map<String, NodeSpec> nodes = Map.ofEntries(
  Map.entry("processName", new NodeSpec(null,"Create Requisition & Collect Approvals",null)),
  Map.entry("start",    new NodeSpec(StartEvent.class,"Start",null)),
  Map.entry("createReq",new NodeSpec(UserTask.class,"Create Purchase Requisition","buyer")),
  Map.entry("fork",     new NodeSpec(ParallelGateway.class,"+",null)),
  Map.entry("mgrApp",   new NodeSpec(UserTask.class,"Manager Approval","buyer")),
  Map.entry("finApp",   new NodeSpec(UserTask.class,"Finance Approval","buyer")),
  Map.entry("join",     new NodeSpec(ParallelGateway.class,"+",null)),
  Map.entry("end",      new NodeSpec(EndEvent.class,"Requisition Approved",null))
);

List<String[]> flows = List.of(
  new String[]{"start","createReq"},
  new String[]{"createReq","fork"},
  new String[]{"fork","mgrApp"},
  new String[]{"fork","finApp"},
  new String[]{"mgrApp","join"},
  new String[]{"finApp","join"},
  new String[]{"join","end"}
);

Map<String,double[]> pos = Map.ofEntries(
  Map.entry("start",     new double[]{40,150}),
  Map.entry("createReq", new double[]{140,150}),
  Map.entry("fork",      new double[]{280,150}),
  Map.entry("mgrApp",    new double[]{420, 60}),
  Map.entry("finApp",    new double[]{420,150}),
  Map.entry("join",      new double[]{560,150}),
  Map.entry("end",       new double[]{700,150})
);

BpmnBuilderStd.generate(
  "FTP_1150_uc9_requisition_approvals",
  "Procurement",
  "Buyer",
  nodes,
  flows,
  pos
);


✅  Saved FTP_1150_uc9_requisition_approvals.bpmn


In [58]:
// UC10 – Notify supplier and record order
Map<String, NodeSpec> nodes = Map.ofEntries(
  Map.entry("processName", new NodeSpec(null,"Notify Supplier & Record Order",null)),
  Map.entry("start",       new NodeSpec(StartEvent.class,"Start",null)),
  Map.entry("inform",      new NodeSpec(UserTask.class,"Inform Supplier","buyer")),
  Map.entry("supplierApp", new NodeSpec(UserTask.class,"Get Supplier Approval","buyer")),
  Map.entry("decide",      new NodeSpec(ExclusiveGateway.class,"?",null)),
  Map.entry("filePO",      new NodeSpec(UserTask.class,"File Purchase Order","buyer")),
  Map.entry("recordPO",    new NodeSpec(UserTask.class,"Record Purchase Order","buyer")),
  Map.entry("confirmDel",  new NodeSpec(UserTask.class,"Confirm Delivery Date","buyer")),
  Map.entry("join",        new NodeSpec(ExclusiveGateway.class,"",null)),
  Map.entry("end",         new NodeSpec(EndEvent.class,"PO Filed",null))
);

List<String[]> flows = List.of(
  new String[]{"start","inform"},
  new String[]{"inform","supplierApp"},
  new String[]{"supplierApp","decide"},
  new String[]{"decide","filePO"},
  new String[]{"decide","recordPO"},
  new String[]{"decide","confirmDel"},
  new String[]{"filePO","join"},
  new String[]{"recordPO","join"},
  new String[]{"confirmDel","join"},
  new String[]{"join","end"}
);

Map<String,double[]> pos = Map.ofEntries(
  Map.entry("start",       new double[]{40,150}),
  Map.entry("inform",      new double[]{140,150}),
  Map.entry("supplierApp", new double[]{280,150}),
  Map.entry("decide",      new double[]{420,150}),
  Map.entry("filePO",      new double[]{560, 60}),
  Map.entry("recordPO",    new double[]{560,150}),
  Map.entry("confirmDel",  new double[]{560,240}),
  Map.entry("join",        new double[]{700,150}),
  Map.entry("end",         new double[]{840,150})
);

BpmnBuilderStd.generate(
  "FTP_1150_uc10_notify_record_order",
  "Procurement",
  "Buyer",
  nodes,
  flows,
  pos
);


✅  Saved FTP_1150_uc10_notify_record_order.bpmn


# IOPM 585

In [59]:
// Employee Orientation process :contentReference[oaicite:1]{index=1}
Map<String, NodeSpec> nodes = Map.ofEntries(
  Map.entry("processName", new NodeSpec(null, "Employee Orientation", null)),
  Map.entry("start",    new NodeSpec(StartEvent.class,      "New Hire Arrived",          null)),
  Map.entry("intro",    new NodeSpec(UserTask.class,       "Make Introduction",          "HR Coordinator")),
  Map.entry("approval", new NodeSpec(UserTask.class,       "Get Approval",               "HR Coordinator")),
  Map.entry("decision", new NodeSpec(ExclusiveGateway.class,"",                          null)),
  Map.entry("tour",     new NodeSpec(UserTask.class,       "Tour Office",                "HR Coordinator")),
  Map.entry("training", new NodeSpec(UserTask.class,       "Assign Training Material",   "HR Coordinator")),
  Map.entry("end",      new NodeSpec(EndEvent.class,       "Checklist Complete",         null))
);

List<String[]> flows = List.of(
  new String[]{"start","intro"},
  new String[]{"intro","approval"},
  new String[]{"approval","decision"},
  new String[]{"decision","tour"},
  new String[]{"decision","training"},
  new String[]{"tour","end"},
  new String[]{"training","end"}
);

Map<String,double[]> pos = Map.ofEntries(
  Map.entry("start",    new double[]{  40,150}),
  Map.entry("intro",    new double[]{ 140,150}),
  Map.entry("approval", new double[]{ 280,150}),
  Map.entry("decision", new double[]{ 420,150}),
  Map.entry("tour",     new double[]{ 560, 70}),
  Map.entry("training", new double[]{ 560,230}),
  Map.entry("end",      new double[]{ 700,150})
);

BpmnBuilderStd.generate(
  "EMP_onboarding_orientation",
  "HR Department",
  "HR Coordinator",
  nodes, flows, pos
);


✅  Saved EMP_onboarding_orientation.bpmn


In [60]:
// Paperwork & 60-Day Roadmap process :contentReference[oaicite:2]{index=2}
Map<String, NodeSpec> nodes = Map.ofEntries(
  Map.entry("processName", new NodeSpec(null, "Paperwork & 60-Day Roadmap", null)),
  Map.entry("start",  new NodeSpec(StartEvent.class, "Begin Compliance",            null)),
  Map.entry("check",  new NodeSpec(UserTask.class,  "Check Paperwork",              "HR Coordinator")),
  Map.entry("review", new NodeSpec(UserTask.class,  "Review 60-Day Plan",           "HR Coordinator")),
  Map.entry("plan",   new NodeSpec(UserTask.class,  "Plan Check-in Meetings",       "HR Coordinator")),
  Map.entry("end",    new NodeSpec(EndEvent.class,  "Roadmap Shared",               null))
);

List<String[]> flows = List.of(
  new String[]{"start","check"},
  new String[]{"check","review"},
  new String[]{"review","plan"},
  new String[]{"plan","end"}
);

Map<String,double[]> pos = Map.ofEntries(
  Map.entry("start",  new double[]{ 40,150}),
  Map.entry("check",  new double[]{140,150}),
  Map.entry("review", new double[]{280,150}),
  Map.entry("plan",   new double[]{420,150}),
  Map.entry("end",    new double[]{560,150})
);

BpmnBuilderStd.generate(
  "EMP_onboarding_paperwork",
  "HR Department",
  "HR Coordinator",
  nodes, flows, pos
);


✅  Saved EMP_onboarding_paperwork.bpmn


In [61]:
// Growth & Social Integration process :contentReference[oaicite:3]{index=3}
Map<String, NodeSpec> nodes = Map.ofEntries(
  Map.entry("processName", new NodeSpec(null, "Growth & Social Integration", null)),
  Map.entry("start",   new NodeSpec(StartEvent.class, "Begin Culture Share",        null)),
  Map.entry("reading", new NodeSpec(UserTask.class,  "Provide Growth Material",     "HR Coordinator")),
  Map.entry("goals",   new NodeSpec(UserTask.class,  "Explain Long-Term Goals",     "HR Coordinator")),
  Map.entry("social",  new NodeSpec(UserTask.class,  "Encourage Social Interaction","HR Coordinator")),
  Map.entry("end",     new NodeSpec(EndEvent.class,  "Culture Understood",          null))
);

List<String[]> flows = List.of(
  new String[]{"start","reading"},
  new String[]{"reading","goals"},
  new String[]{"goals","social"},
  new String[]{"social","end"}
);

Map<String,double[]> pos = Map.ofEntries(
  Map.entry("start",   new double[]{ 40,150}),
  Map.entry("reading", new double[]{140,150}),
  Map.entry("goals",   new double[]{280,150}),
  Map.entry("social",  new double[]{420,150}),
  Map.entry("end",     new double[]{560,150})
);

BpmnBuilderStd.generate(
  "EMP_onboarding_growth",
  "HR Department",
  "HR Coordinator",
  nodes, flows, pos
);


✅  Saved EMP_onboarding_growth.bpmn


In [62]:
// Assess & Redesign process :contentReference[oaicite:4]{index=4}
Map<String, NodeSpec> nodes = Map.ofEntries(
  Map.entry("processName", new NodeSpec(null, "Assess & Redesign", null)),
  Map.entry("start",     new NodeSpec(StartEvent.class, "Start Assessment",            null)),
  Map.entry("assess",    new NodeSpec(UserTask.class,  "Assess Performance",           "Process Analyst")),
  Map.entry("identify",  new NodeSpec(UserTask.class,  "Identify Components",          "Process Analyst")),
  Map.entry("confirm",   new NodeSpec(UserTask.class,  "Confirm Optimisation Need",    "Process Analyst")),
  Map.entry("automate",  new NodeSpec(UserTask.class,  "Note Automatable Steps",       "Process Analyst")),
  Map.entry("duration",  new NodeSpec(UserTask.class,  "Define Duration",              "Process Analyst")),
  Map.entry("improve",   new NodeSpec(UserTask.class,  "Write Improvements",           "Process Analyst")),
  Map.entry("end",       new NodeSpec(EndEvent.class,  "Redesign Draft Ready",         null))
);

List<String[]> flows = List.of(
  new String[]{"start","assess"},
  new String[]{"assess","identify"},
  new String[]{"identify","confirm"},
  new String[]{"confirm","automate"},
  new String[]{"automate","duration"},
  new String[]{"duration","improve"},
  new String[]{"improve","end"}
);

Map<String,double[]> pos = Map.ofEntries(
  Map.entry("start",    new double[]{ 40,150}),
  Map.entry("assess",   new double[]{140,150}),
  Map.entry("identify", new double[]{280,150}),
  Map.entry("confirm",  new double[]{420,150}),
  Map.entry("automate", new double[]{560,150}),
  Map.entry("duration", new double[]{700,150}),
  Map.entry("improve",  new double[]{840,150}),
  Map.entry("end",      new double[]{980,150})
);

BpmnBuilderStd.generate(
  "PROC_assess_redesign",
  "Process Excellence",
  "Process Analyst",
  nodes, flows, pos
);


✅  Saved PROC_assess_redesign.bpmn


In [63]:
// Roll-out & Monitor process :contentReference[oaicite:5]{index=5}
Map<String, NodeSpec> nodes = Map.ofEntries(
  Map.entry("processName", new NodeSpec(null, "Roll-out & Monitor", null)),
  Map.entry("start",      new NodeSpec(StartEvent.class, "Start Roll-out",              null)),
  Map.entry("approval",   new NodeSpec(UserTask.class,  "Get Improvement Approval",     "Process Analyst")),
  Map.entry("implement",  new NodeSpec(UserTask.class,  "Implement Changes",            "Process Analyst")),
  Map.entry("communicate",new NodeSpec(UserTask.class,  "Communicate Changes",          "Process Analyst")),
  Map.entry("monitor",    new NodeSpec(UserTask.class,  "Determine Monitoring Method",  "Process Analyst")),
  Map.entry("review",     new NodeSpec(UserTask.class,  "Schedule Next Review",         "Process Analyst")),
  Map.entry("end",        new NodeSpec(EndEvent.class,  "Optimisation Cycle Active",    null))
);

List<String[]> flows = List.of(
  new String[]{"start","approval"},
  new String[]{"approval","implement"},
  new String[]{"implement","communicate"},
  new String[]{"communicate","monitor"},
  new String[]{"monitor","review"},
  new String[]{"review","end"}
);

Map<String,double[]> pos = Map.ofEntries(
  Map.entry("start",      new double[]{ 40,150}),
  Map.entry("approval",   new double[]{140,150}),
  Map.entry("implement",  new double[]{280,150}),
  Map.entry("communicate",new double[]{420,150}),
  Map.entry("monitor",    new double[]{560,150}),
  Map.entry("review",     new double[]{700,150}),
  Map.entry("end",        new double[]{840,150})
);

BpmnBuilderStd.generate(
  "PROC_rollout_monitor",
  "Process Excellence",
  "Process Analyst",
  nodes, flows, pos
);


✅  Saved PROC_rollout_monitor.bpmn


# IOPM 855

In [64]:
// ── UC1  Welcome new hire ─────────────────────────────────────────────
Map<String,NodeSpec> nodes = Map.ofEntries(
  Map.entry("processName", new NodeSpec(null,                "Welcome new hire",             null)),
  Map.entry("start",       new NodeSpec(StartEvent.class,    "Day 1 begins",                 null)),
  Map.entry("intro",       new NodeSpec(UserTask.class,      "Make introduction",            "HR Coordinator")),
  Map.entry("prepare",     new NodeSpec(UserTask.class,      "Prepare employee paperwork",   "HR Coordinator")),
  Map.entry("lunch",       new NodeSpec(UserTask.class,      "Take hire to lunch",           "HR Coordinator")),
  Map.entry("end",         new NodeSpec(EndEvent.class,      "Welcome complete",             null))
);

List<String[]> flows = List.of(
  new String[]{"start","intro"},
  new String[]{"intro","prepare"},
  new String[]{"prepare","lunch"},
  new String[]{"lunch","end"}
);

Map<String,double[]> pos = Map.ofEntries(
  Map.entry("start",   new double[]{ 40,150}),
  Map.entry("intro",   new double[]{140,150}),
  Map.entry("prepare", new double[]{280,150}),
  Map.entry("lunch",   new double[]{420,150}),
  Map.entry("end",     new double[]{560,150})
);

BpmnBuilderStd.generate(
  "UC1_welcome_new_hire",
  "Employee Onboarding",
  "HR Coordinator",
  nodes, flows, pos
);


✅  Saved UC1_welcome_new_hire.bpmn


In [65]:
// ── UC2  Set first-month goals ────────────────────────────────────────
Map<String,NodeSpec> nodes = Map.ofEntries(
  Map.entry("processName", new NodeSpec(null,                "Set first-month goals",        null)),
  Map.entry("start",       new NodeSpec(StartEvent.class,    "Begin goal-setting",           null)),
  Map.entry("explain",     new NodeSpec(UserTask.class,      "Explain expectations",         "HR Coordinator")),
  Map.entry("approve",     new NodeSpec(UserTask.class,      "Get approval",                 "HR Coordinator")),
  Map.entry("end",         new NodeSpec(EndEvent.class,      "Goals documented",             null))
);

List<String[]> flows = List.of(
  new String[]{"start","explain"},
  new String[]{"explain","approve"},
  new String[]{"approve","end"}
);

Map<String,double[]> pos = Map.ofEntries(
  Map.entry("start",   new double[]{ 40,150}),
  Map.entry("explain", new double[]{140,150}),
  Map.entry("approve", new double[]{280,150}),
  Map.entry("end",     new double[]{420,150})
);

BpmnBuilderStd.generate(
  "UC2_set_first_month_goals",
  "Employee Onboarding",
  "HR Coordinator",
  nodes, flows, pos
);


✅  Saved UC2_set_first_month_goals.bpmn


In [66]:
// ── UC3  Activate first assignments ──────────────────────────────────
Map<String,NodeSpec> nodes = Map.ofEntries(
  Map.entry("processName", new NodeSpec(null,                "Activate first assignments",   null)),
  Map.entry("start",       new NodeSpec(StartEvent.class,    "Activation trigger",           null)),
  Map.entry("split",       new NodeSpec(ParallelGateway.class,"",                             null)),
  Map.entry("assign",      new NodeSpec(UserTask.class,      "Assign first project",         "HR Coordinator")),
  Map.entry("check",       new NodeSpec(UserTask.class,      "Check paperwork",              "HR Coordinator")),
  Map.entry("material",    new NodeSpec(UserTask.class,      "Provide growth material",      "HR Coordinator")),
  Map.entry("join",        new NodeSpec(ParallelGateway.class,"",                             null)),
  Map.entry("end",         new NodeSpec(EndEvent.class,      "Ready to contribute",          null))
);

List<String[]> flows = List.of(
  new String[]{"start","split"},
  new String[]{"split","assign"},
  new String[]{"split","check"},
  new String[]{"split","material"},
  new String[]{"assign","join"},
  new String[]{"check","join"},
  new String[]{"material","join"},
  new String[]{"join","end"}
);

Map<String,double[]> pos = Map.ofEntries(
  Map.entry("start",    new double[]{ 40,150}),
  Map.entry("split",    new double[]{140,150}),
  Map.entry("assign",   new double[]{280, 70}),
  Map.entry("check",    new double[]{280,150}),
  Map.entry("material", new double[]{280,230}),
  Map.entry("join",     new double[]{420,150}),
  Map.entry("end",      new double[]{560,150})
);

BpmnBuilderStd.generate(
  "UC3_activate_first_assignments",
  "Employee Onboarding",
  "HR Coordinator",
  nodes, flows, pos
);


✅  Saved UC3_activate_first_assignments.bpmn


In [67]:
// ── UC4  Evaluate process efficiency ─────────────────────────────────
Map<String,NodeSpec> nodes = Map.ofEntries(
  Map.entry("processName", new NodeSpec(null,                "Evaluate process efficiency",  null)),
  Map.entry("start",       new NodeSpec(StartEvent.class,    "Evaluation starts",            null)),
  Map.entry("confirm",     new NodeSpec(UserTask.class,      "Confirm optimisation need",    "Process Analyst")),
  Map.entry("assess",      new NodeSpec(UserTask.class,      "Assess process performance",   "Process Analyst")),
  Map.entry("examine",     new NodeSpec(UserTask.class,      "Examine wasteful steps",       "Process Analyst")),
  Map.entry("improve",     new NodeSpec(UserTask.class,      "Write potential improvements", "Process Analyst")),
  Map.entry("end",         new NodeSpec(EndEvent.class,      "Proposal ready",               null))
);

List<String[]> flows = List.of(
  new String[]{"start","confirm"},
  new String[]{"confirm","assess"},
  new String[]{"assess","examine"},
  new String[]{"examine","improve"},
  new String[]{"improve","end"}
);

Map<String,double[]> pos = Map.ofEntries(
  Map.entry("start",   new double[]{ 40,150}),
  Map.entry("confirm", new double[]{140,150}),
  Map.entry("assess",  new double[]{280,150}),
  Map.entry("examine", new double[]{420,150}),
  Map.entry("improve", new double[]{560,150}),
  Map.entry("end",     new double[]{700,150})
);

BpmnBuilderStd.generate(
  "UC4_evaluate_process_efficiency",
  "Process Improvement",
  "Process Analyst",
  nodes, flows, pos
);


✅  Saved UC4_evaluate_process_efficiency.bpmn


In [68]:
// ── UC5  Deploy updated process ──────────────────────────────────────
Map<String,NodeSpec> nodes = Map.ofEntries(
  Map.entry("processName", new NodeSpec(null,                "Deploy updated process",       null)),
  Map.entry("start",       new NodeSpec(StartEvent.class,    "Deployment starts",            null)),
  Map.entry("approval",    new NodeSpec(UserTask.class,      "Get approval",                 "Process Analyst")),
  Map.entry("useUpdated",  new NodeSpec(UserTask.class,      "Use updated process",          "Process Analyst")),
  Map.entry("implement",   new NodeSpec(UserTask.class,      "Implement changes",            "Process Analyst")),
  Map.entry("monitor",     new NodeSpec(UserTask.class,      "Determine monitoring method",  "Process Analyst")),
  Map.entry("schedule",    new NodeSpec(UserTask.class,      "Schedule next review",         "Process Analyst")),
  Map.entry("end",         new NodeSpec(EndEvent.class,      "Process operational",          null))
);

List<String[]> flows = List.of(
  new String[]{"start","approval"},
  new String[]{"approval","useUpdated"},
  new String[]{"useUpdated","implement"},
  new String[]{"implement","monitor"},
  new String[]{"monitor","schedule"},
  new String[]{"schedule","end"}
);

Map<String,double[]> pos = Map.ofEntries(
  Map.entry("start",      new double[]{ 40,150}),
  Map.entry("approval",   new double[]{140,150}),
  Map.entry("useUpdated", new double[]{280,150}),
  Map.entry("implement",  new double[]{420,150}),
  Map.entry("monitor",    new double[]{560,150}),
  Map.entry("schedule",   new double[]{700,150}),
  Map.entry("end",        new double[]{840,150})
);

BpmnBuilderStd.generate(
  "UC5_deploy_updated_process",
  "Process Improvement",
  "Process Analyst",
  nodes, flows, pos
);


✅  Saved UC5_deploy_updated_process.bpmn
